In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [246]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import unicodedata
import re

In [ ]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches").text
soup = BeautifulSoup(html)

In [ ]:
h3 = soup.find_all("h3")

In [ ]:
if not("tables") in dir(): tables = {}

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
html = requests.get(static_url).text
soup = BeautifulSoup(html)


In [4]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [5]:
html_tables = soup.find_all("table")
html_tables = html_tables[2:]


In [233]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=' '.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)
    """
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name
    """
    if np.logical_and(np.logical_and(not(colunm_name.strip().isdigit()), not(colunm_name==" ")), not(colunm_name=="")):
      colunm_name = colunm_name.strip()
      return colunm_name

In [7]:
# task2
colunm_name = []
for row in html_tables[0].find("tr").find_all("th"):
  name = extract_column_from_header(row)
  if name != "":
    colunm_name.append(name)


In [301]:
#task 3
launch_dict = dict.fromkeys(colunm_name)
del launch_dict["Date and time ( )"]
for k in launch_dict.keys():
  launch_dict[k] = []
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [251]:
launch_dict.keys()

dict_keys(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'])

In [302]:
extracted_row = 0
for table_number,table in enumerate(soup.find_all('table',attrs={"class":"wikitable plainrowheaders collapsible"})):
  #print(table)
  for rows in table.find_all("tr"):

    if rows.th:
      if rows.th.string:
        flight_number = rows.th.string.strip()
        flag = flight_number.isdigit()
    else:
      flag = False

    row = rows.find_all("td")
    if flag:
      extracted_row += 1
      launch_dict["Flight No."].append(flight_number)
      datatimelist=date_time(row[0])
      launch_dict["Date"].append(datatimelist[0].strip(","))
      launch_dict["Time"].append(datatimelist[1])

      bv = booster_version(row[1])
      if not(bv):
        bv = row[1].a.string

      launch_dict["Version Booster"].append(bv)

      launch_site = row[2].a.string
      launch_dict["Launch site"].append(launch_site)

      payload = row[3].a.string
      launch_dict["Payload"].append(launch_site)

      mass = get_mass(row[4])
      launch_dict["Payload mass"].append(mass)

      orbit = row[5].a.string
      launch_dict["Orbit"].append(orbit)

      if row[6].a:
        customer = row[6].a.string
      else:
        customer = row[6].string.strip()
      launch_dict["Customer"].append(customer)

      launch_outcome = list(row[7].strings)[0].strip()
      launch_dict["Launch outcome"].append(launch_outcome)

      booster_landing = landing_status(row[8]).strip()
      launch_dict["Booster landing"].append(booster_landing)




In [304]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [306]:
df.to_csv('spacex_web_scraped.csv', index=False)